In [ ]:
import torch 
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import random

In [0]:
dir_name = "/content/train/" #Path to images directory
label_0_dir = dir_name + "0/"
label_1_dir = dir_name + "1/"
models_dir = "/content/modelsRes18/" #Path to save models

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [0]:
model18 = models.resnet18(pretrained=True)

In [0]:
model = model18

In [0]:
for layer, param in model.named_parameters():
    
  if 'layer4' not in layer:
    param.requires_grad = False
    
model.fc = torch.nn.Linear(model.fc.in_features, 2)

In [0]:
train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ])

In [44]:
dataset = datasets.ImageFolder(dir_name, transform = train_transforms)

dataset_size = len(dataset)
train_size = int(dataset_size * 0.6)
val_size = int(dataset_size * 0.2)
test_size = dataset_size - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

print('Dataset size: ', len(dataset))
print('Train set size: ', len(train_dataset))
print('Validation set size: ', len(val_dataset))
print('Test set size: ', len(test_dataset))

Dataset size:  3788
Train set size:  2272
Validation set size:  757
Test set size:  759


In [0]:
BATCH_SIZE = 20

train_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)

val_loader = torch.utils.data.DataLoader(val_dataset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)

In [0]:
LEARNING_RATE = 0.001

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [47]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [48]:
total_epoch = 20
best_epoch = 0
training_losses = []
val_losses = []

for epoch in range(total_epoch):
    
    epoch_train_loss = 0
    for X, y in train_loader:
        
        X, y = X.cuda(), y.cuda()
        
        optimizer.zero_grad()
        result = model(X)
        loss = criterion(result, y)
        epoch_train_loss += loss.item()
        loss.backward()
        optimizer.step()
      
    training_losses.append(epoch_train_loss)
    
    
    epoch_val_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for X, y in val_loader:
            
            X, y = X.cuda(), y.cuda()
             
            result = model(X)
            loss = criterion(result, y)
            epoch_val_loss += loss.item()
            _, maximum = torch.max(result.data, 1)
            total += y.size(0)
            correct += (maximum == y).sum().item()
            
    val_losses.append(epoch_val_loss)
    accuracy = correct/total
    print("EPOCH:", epoch, ", Training Loss:", epoch_train_loss, ", Validation Loss:", epoch_val_loss, ", Accuracy: ", accuracy)
    
    
    if min(val_losses) == val_losses[-1]:
        best_epoch = epoch
        checkpoint = {'model': model,
                            'state_dict': model.state_dict(),
                            'optimizer' : optimizer.state_dict()}

        torch.save(checkpoint, models_dir + '{}.pth'.format(epoch))
        print("Model saved")

EPOCH: 0 , Training Loss: 5.157025273016188 , Validation Loss: 0.5413916584220715 , Accuracy:  0.9973579920739762
Model saved
EPOCH: 1 , Training Loss: 1.00443336526223 , Validation Loss: 1.3244673720328137 , Accuracy:  0.9867899603698811
EPOCH: 2 , Training Loss: 0.7806696451370954 , Validation Loss: 1.023388894594973 , Accuracy:  0.9894319682959049
EPOCH: 3 , Training Loss: 1.0875125254478917 , Validation Loss: 1.2635807456972543 , Accuracy:  0.9907529722589168
EPOCH: 4 , Training Loss: 1.079637612307124 , Validation Loss: 1.2519135597940476 , Accuracy:  0.9907529722589168
EPOCH: 5 , Training Loss: 0.5180802304403187 , Validation Loss: 1.492515321982637 , Accuracy:  0.9933949801849405
EPOCH: 6 , Training Loss: 0.2549463428067611 , Validation Loss: 1.1912097698764228 , Accuracy:  0.9907529722589168
EPOCH: 7 , Training Loss: 1.2711125967730368 , Validation Loss: 1.2138547142822063 , Accuracy:  0.9920739762219286
EPOCH: 8 , Training Loss: 0.2062683579615623 , Validation Loss: 0.57727982